# Dataset Construction: Energy Demand and Supply In Spain

### Demand Load & Forecast Cleaning

Steps taken to prepare the energy demand and energy demand forecast into one dataframe.

In [1]:
import pandas as pd

In [2]:
#inspect the files to be merged and cleaned
!ls -l ./raw/demand_load_forecast

total 4848
-rw-r--r--@ 1 ns  staff  473123 18 Aug 20:03 Total Load - Day Ahead _ Actual_2015.csv
-rw-r--r--@ 1 ns  staff  474463 18 Aug 20:03 Total Load - Day Ahead _ Actual_2016.csv
-rw-r--r--@ 1 ns  staff  473173 18 Aug 20:03 Total Load - Day Ahead _ Actual_2017.csv
-rw-r--r--@ 1 ns  staff  473173 18 Aug 20:00 Total Load - Day Ahead _ Actual_2018.csv
-rw-r--r--@ 1 ns  staff  449407 29 Aug 17:59 Total Load - Day Ahead _ Actual_2019.csv


In [17]:
#load the files
path = './raw/demand_load_forecast/'
files = ['Total Load - Day Ahead _ Actual_2015.csv',
            'Total Load - Day Ahead _ Actual_2016.csv',
            'Total Load - Day Ahead _ Actual_2017.csv',
            'Total Load - Day Ahead _ Actual_2018.csv',
            'Total Load - Day Ahead _ Actual_2019.csv']

dataset = [pd.read_csv(path+file) for file in files]
data = dataset.copy()

In [18]:
#inspect the first list pandas element 
data[0].head(3)

,Time (CET),Day-ahead Total Load Forecast [MW] - BZN|ES,Actual Total Load [MW] - BZN|ES
0,01.01.2015 00:00 - 01.01.2015 01:00,26118.0,25385.0
1,01.01.2015 01:00 - 01.01.2015 02:00,24934.0,24382.0
2,01.01.2015 02:00 - 01.01.2015 03:00,23515.0,22734.0


In [19]:
def format_load_forecast_data(data):
    '''
    Input: A dataframe of Day Ahead Total Load, and Actual Load obtained from csv data obtained from the entsoe Transparency Platform.
    
    Descrption:
    Input is a 3 column dataframe consisting of text time stamps with hourly frequency. 
    - Function formats the string in order to be formatted into a datetime.
    - Appends a datetime index and drops the time strings
    
    Output: A 2 column dataframe with a DatetimeIndex
    
    '''
    
    #set column names to something simple
    data.columns = ['time', 'day_forecast',
       'actual_load']

    #set the time to the first element in the time string. 
    #So 01.01.2018 00:00 - 01.01.2018 01:00 becomes 01.01.2018 00:00
    data['time'] = data['time'].str.split('-').apply(lambda x: x[0]).str.strip()
     
    #set the time strings to datetime obejects and set index as date time
    datetimes = pd.to_datetime(data['time'], format='%d-%m-%Y %H%M', errors='ignore')
    data_ = data.set_index(pd.DatetimeIndex(datetimes))
    
    #remove extra time column with original string objects
    data_time = data_[['day_forecast', 'actual_load']]
    
    return data_time


#get the numberof input files processed
years = range(len(files))

#create a dictionary of formatted pandas dataframes where key is each year
data_formatted = {year: format_load_forecast_data(data) for year,data in zip(years, data)}

In [20]:
def combine_annual_data(dictionary):
    """
    Input: a dictionary of dataframes.
    
    Output: a single dataframe
    """
    
    all_data_list = []
    
    for key in dictionary.keys():
        all_data_list.append(dictionary[key])
        
    data_all_years = pd.concat(all_data_list)
    
    return data_all_years

data = combine_annual_data(data_formatted)

In [21]:
data.head()

,day_forecast,actual_load
time,,
2015-01-01 00:00:00,26118,25385
2015-01-01 01:00:00,24934,24382
2015-01-01 02:00:00,23515,22734
2015-01-01 03:00:00,22642,21286
2015-01-01 04:00:00,21785,20264


In [22]:
#export the data to the processed file
actual_demand_and_forecast = data.copy()

save_path = './processed/'
actual_demand_and_forecast.to_csv(save_path + 'Actual-demand-forecast-demand-2015-2019.csv')

In [23]:
!ls -l ./processed

total 2968
-rw-r--r--  1 ns  staff  1518679  7 Oct 16:18 Actual-demand-forecast-demand-2015-2019.csv


### Generation Source Cleaning



In [26]:
#inspect the files to be merged and cleaned
!ls -l ./raw/generation

total 18080
-rw-r--r--@ 1 ns  staff  1456154  7 Oct 16:26 Actual Generation per Production Type_2015-2016.csv
-rw-r--r--@ 1 ns  staff  1458242  7 Oct 15:28 Actual Generation per Production Type_2016-2017.csv
-rw-r--r--@ 1 ns  staff  1458263  7 Oct 15:28 Actual Generation per Production Type_2017-2018.csv
-rw-r--r--@ 1 ns  staff  1459284  7 Oct 15:27 Actual Generation per Production Type_2018-2019.csv
-rw-r--r--@ 1 ns  staff  1388375  7 Oct 15:27 Actual Generation per Production Type_2019-2020.csv


In [73]:
#load the files for generation
path = './raw/generation/'
files = ['Actual Generation per Production Type_2015-2016.csv',
         'Actual Generation per Production Type_2016-2017.csv',
        'Actual Generation per Production Type_2017-2018.csv',
            'Actual Generation per Production Type_2018-2019.csv',
            'Actual Generation per Production Type_2019-2020.csv']

generation_dataset = [pd.read_csv(path+file) for file in files]
data_gen = generation_dataset.copy()

In [57]:
data_gen[0].head(3)

,Area,MTU,Biomass - Actual Aggregated [MW],Fossil Brown coal/Lignite - Actual Aggregated [MW],Fossil Coal-derived gas - Actual Aggregated [MW],Fossil Gas - Actual Aggregated [MW],Fossil Hard coal - Actual Aggregated [MW],Fossil Oil - Actual Aggregated [MW],Fossil Oil shale - Actual Aggregated [MW],Fossil Peat - Actual Aggregated [MW],...,Hydro Run-of-river and poundage - Actual Aggregated [MW],Hydro Water Reservoir - Actual Aggregated [MW],Marine - Actual Aggregated [MW],Nuclear - Actual Aggregated [MW],Other - Actual Aggregated [MW],Other renewable - Actual Aggregated [MW],Solar - Actual Aggregated [MW],Waste - Actual Aggregated [MW],Wind Offshore - Actual Aggregated [MW],Wind Onshore - Actual Aggregated [MW]
0,BZN|ES,01.01.2015 00:00 - 01.01.2015 01:00 (CET),447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,...,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0
1,BZN|ES,01.01.2015 01:00 - 01.01.2015 02:00 (CET),449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,...,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0
2,BZN|ES,01.01.2015 02:00 - 01.01.2015 03:00 (CET),448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,...,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0


In [58]:
data_gen[0].columns.str.lower()

Index(['area', 'mtu', 'biomass  - actual aggregated [mw]',
       'fossil brown coal/lignite  - actual aggregated [mw]',
       'fossil coal-derived gas  - actual aggregated [mw]',
       'fossil gas  - actual aggregated [mw]',
       'fossil hard coal  - actual aggregated [mw]',
       'fossil oil  - actual aggregated [mw]',
       'fossil oil shale  - actual aggregated [mw]',
       'fossil peat  - actual aggregated [mw]',
       'geothermal  - actual aggregated [mw]',
       'hydro pumped storage  - actual aggregated [mw]',
       'hydro pumped storage  - actual consumption [mw]',
       'hydro run-of-river and poundage  - actual aggregated [mw]',
       'hydro water reservoir  - actual aggregated [mw]',
       'marine  - actual aggregated [mw]', 'nuclear  - actual aggregated [mw]',
       'other  - actual aggregated [mw]',
       'other renewable  - actual aggregated [mw]',
       'solar  - actual aggregated [mw]', 'waste  - actual aggregated [mw]',
       'wind offshore  - actual 

In [74]:
data = data_gen.copy()

def format_generation_data(data):
    #set column names to something simple
    data.columns = ['area', 
                    'time', 
                    'biomass', 
                    'fossil brown coal/lignite', 
                    'fossil coal-derived gas',
                    'fossil gas',
                    'fossil hard coal',
                    'fossil oil',
                    'fossil oil shale',
                    'fossil peat',
                    'geothermal',
                    'hydro pumped storage',
                    'hydro pumped storage',
                    'hydro run-of-river and poundage',
                    'hydro water reservoir',
                    'marine', 
                    'nuclear',
                    'other',
                    'other renewable',
                    'solar', 
                    'waste',
                    'wind offshore',
                    'wind onshore']

    #set the time to the first element in the time string. 
    #So 01.01.2018 00:00 - 01.01.2018 01:00 becomes 01.01.2018 00:00
    data['time'] = data['time'].str.split('-').apply(lambda x: x[0]).str.strip()

    #set the time strings to datetime obejects and set index as date time
    datetimes = pd.to_datetime(data['time'], format='%d-%m-%Y %H%M', errors='ignore')
    data_ = data.set_index(pd.DatetimeIndex(datetimes))

    #remove area column and time string columns
    data_.drop(['area', 'time'], axis=1, inplace=True)

    return data_


#get the numberof input files processed
years = range(len(files))

#create a dictionary of formatted pandas dataframes where key is each year
data_generation = {year: format_generation_data(data) for year,data in zip(years, data_gen)}

In [75]:
#check the output
data_generation_all = combine_annual_data(data_generation)

data_generation_all.head(3)

,biomass,fossil brown coal/lignite,fossil coal-derived gas,fossil gas,fossil hard coal,fossil oil,fossil oil shale,fossil peat,geothermal,hydro pumped storage,...,hydro run-of-river and poundage,hydro water reservoir,marine,nuclear,other,other renewable,solar,waste,wind offshore,wind onshore
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,447,329,0,4844,4821,162,0,0,0,NaN,...,1051,1899,0,7096,43,73,49,196,0,6378
2015-01-01 01:00:00,449,328,0,5196,4755,158,0,0,0,NaN,...,1009,1658,0,7096,43,71,50,195,0,5890
2015-01-01 02:00:00,448,323,0,4857,4581,157,0,0,0,NaN,...,973,1371,0,7099,43,73,50,196,0,5461


In [76]:
data_generation_all.to_csv(save_path + 'generation-production-type-2015-2019.csv')

In [65]:
!ls -l ./processed

total 13384
-rw-r--r--  1 ns  staff  1518679  7 Oct 16:18 Actual-demand-forecast-demand-2015-2019.csv
-rw-r--r--  1 ns  staff  5329580  7 Oct 16:48 generation-production-type-2015-2019.csv


### Solar and Wind Generation Forecasts

In [68]:
!ls -l ./raw/generation_forecasts_wind_solar

total 8008
-rw-r--r--@ 1 ns  staff  816253  7 Oct 15:47 Generation Forecasts for Wind and Solar_201501010000-201601010000.csv
-rw-r--r--@ 1 ns  staff  818240  7 Oct 15:48 Generation Forecasts for Wind and Solar_201601010000-201701010000.csv
-rw-r--r--@ 1 ns  staff  816705  7 Oct 15:48 Generation Forecasts for Wind and Solar_201701010000-201801010000.csv
-rw-r--r--@ 1 ns  staff  834281  7 Oct 15:50 Generation Forecasts for Wind and Solar_201801010000-201901010000.csv
-rw-r--r--@ 1 ns  staff  804446  7 Oct 18:00 Generation Forecasts for Wind and Solar_201901010000-202001010000.csv


In [85]:
#load the files for generation
path = './raw/generation_forecasts_wind_solar/'
files = ['Generation Forecasts for Wind and Solar_201501010000-201601010000.csv',
         'Generation Forecasts for Wind and Solar_201601010000-201701010000.csv',
        'Generation Forecasts for Wind and Solar_201701010000-201801010000.csv',
            'Generation Forecasts for Wind and Solar_201801010000-201901010000.csv',
            'Generation Forecasts for Wind and Solar_201901010000-202001010000.csv']

wind_solar_forecast_dataset = [pd.read_csv(path+file) for file in files]
data_gen_forecasts = wind_solar_forecast_dataset.copy()

In [86]:
data_gen_forecasts[0].head(3)

,MTU (CET),Generation - Solar [MW] Day Ahead/ BZN|ES,Generation - Solar [MW] Intraday / BZN|ES,Generation - Solar [MW] Current / BZN|ES,Generation - Wind Offshore [MW] Day Ahead/ BZN|ES,Generation - Wind Offshore [MW] Intraday / BZN|ES,Generation - Wind Offshore [MW] Current / BZN|ES,Generation - Wind Onshore [MW] Day Ahead/ BZN|ES,Generation - Wind Onshore [MW] Intraday / BZN|ES,Generation - Wind Onshore [MW] Current / BZN|ES
0,01.01.2015 00:00 - 01.01.2015 01:00,17.0,n/e,n/e,n/e,n/e,n/e,6436.0,n/e,n/e
1,01.01.2015 01:00 - 01.01.2015 02:00,16.0,n/e,n/e,n/e,n/e,n/e,5856.0,n/e,n/e
2,01.01.2015 02:00 - 01.01.2015 03:00,8.0,n/e,n/e,n/e,n/e,n/e,5454.0,n/e,n/e


In [87]:
data_gen_forecasts[0].columns.str.lower()

Index(['mtu (cet)', 'generation - solar  [mw] day ahead/ bzn|es',
       'generation - solar  [mw] intraday / bzn|es',
       'generation - solar  [mw] current / bzn|es',
       'generation - wind offshore  [mw] day ahead/ bzn|es',
       'generation - wind offshore  [mw] intraday / bzn|es',
       'generation - wind offshore  [mw] current / bzn|es',
       'generation - wind onshore  [mw] day ahead/ bzn|es',
       'generation - wind onshore  [mw] intraday / bzn|es',
       'generation - wind onshore  [mw] current / bzn|es'],
      dtype='object')

In [90]:
data_sw_forecast = data_gen_forecasts.copy()

def format_solar_wind_forecast_data(data):
    #set column names to something simple
    data.columns = ['time', 'solar day ahead',
       'solar intraday',
       'solar current',
       'wind offshore eday ahead',
       'wind offshore intraday',
       'wind offshore current',
       'wind onshore day ahead',
       'wind onshore intraday',
       'wind onshore current']

    #set the time to the first element in the time string. 
    #So 01.01.2018 00:00 - 01.01.2018 01:00 becomes 01.01.2018 00:00
    data['time'] = data['time'].str.split('-').apply(lambda x: x[0]).str.strip()

    #set the time strings to datetime obejects and set index as date time
    datetimes = pd.to_datetime(data['time'], format='%d-%m-%Y %H%M', errors='ignore')
    data_ = data.set_index(pd.DatetimeIndex(datetimes))

    #remove area column and time string columns
    data_.drop(['time'], axis=1, inplace=True)

    return data_


#get the numberof input files processed
years = range(len(files))

#create a dictionary of formatted pandas dataframes where key is each year
data_solar_wind_forecast = {year: format_solar_wind_forecast_data(data) for year,data in zip(years, data_sw_forecast)}

In [91]:
#check the output
data_wind_solar_15_19 = combine_annual_data(data_solar_wind_forecast)

data_wind_solar_15_19.head(3)

,solar day ahead,solar intraday,solar current,wind offshore eday ahead,wind offshore intraday,wind offshore current,wind onshore day ahead,wind onshore intraday,wind onshore current
time,,,,,,,,,
2015-01-01 00:00:00,17,n/e,n/e,n/e,n/e,n/e,6436,n/e,n/e
2015-01-01 01:00:00,16,n/e,n/e,n/e,n/e,n/e,5856,n/e,n/e
2015-01-01 02:00:00,8,n/e,n/e,n/e,n/e,n/e,5454,n/e,n/e


In [93]:
drop_cols = ['solar intraday', 
             'solar current', 
             'wind offshore intraday', 
             'wind offshore current', 
             'wind onshore intraday', 
             'wind onshore current']

data_wind_solar_15_19.drop(drop_cols, axis=1, inplace=True)
data_wind_solar_15_19.head(3)

,solar day ahead,wind offshore eday ahead,wind onshore day ahead
time,,,
2015-01-01 00:00:00,17,n/e,6436
2015-01-01 01:00:00,16,n/e,5856
2015-01-01 02:00:00,8,n/e,5454


In [95]:
data_wind_solar_15_19.to_csv(save_path + 'solar_wind_forecasts.csv')

In [96]:
!ls -l ./processed

total 18360
-rw-r--r--  1 ns  staff  1518679  7 Oct 16:18 Actual-demand-forecast-demand-2015-2019.csv
-rw-r--r--  1 ns  staff  5329580  7 Oct 18:04 generation-production-type-2015-2019.csv
-rw-r--r--  1 ns  staff  1583537  7 Oct 18:16 solar_wind_forecasts.csv


### Weather Source Cleaning

In [97]:
!ls -l ./raw/weather/

total 77968
-rw-r--r--  1 ns  staff  39576929 29 Aug 10:52 weather_2013_2019.csv


In [112]:
path = './raw/weather/weather_2013_2019.csv'
weather_data = pd.read_csv(path)
weather_data.head(3)

,Unnamed: 0,dt,dt_iso,city_id,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,0,2013-10-01 02:00:00,2013-10-01 00:00:00 +0000 UTC,2509954,Valencia,299.150,299.150,299.150,1008,61,5,290,0.0,0.0,0.0,20,801,clouds,few clouds,02n
1,1,2013-10-01 03:00:00,2013-10-01 01:00:00 +0000 UTC,2509954,Valencia,298.150,298.150,298.150,1009,65,4,250,0.0,0.0,0.0,20,801,clouds,few clouds,02n
2,2,2013-10-01 04:00:00,2013-10-01 02:00:00 +0000 UTC,2509954,Valencia,296.161,296.161,296.161,1009,71,4,269,0.0,0.0,0.0,10,800,clear,sky is clear,02


In [109]:
weather_data.head(3)

,Unnamed: 0,dt,dt_iso,city_id,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,0,2013-10-01 02:00:00,2013-10-01 00:00:00 +0000 UTC,2509954,Valencia,299.150,299.150,299.150,1008,61,5,290,0.0,0.0,0.0,20,801,clouds,few clouds,02n
1,1,2013-10-01 03:00:00,2013-10-01 01:00:00 +0000 UTC,2509954,Valencia,298.150,298.150,298.150,1009,65,4,250,0.0,0.0,0.0,20,801,clouds,few clouds,02n
2,2,2013-10-01 04:00:00,2013-10-01 02:00:00 +0000 UTC,2509954,Valencia,296.161,296.161,296.161,1009,71,4,269,0.0,0.0,0.0,10,800,clear,sky is clear,02


In [113]:
datetimes = pd.to_datetime(weather_data['dt'], format='%d-%m-%Y %H%M', errors='ignore')
weather_data = weather_data.set_index(pd.DatetimeIndex(datetimes))
weather_data.drop(['Unnamed: 0', 'dt', 'dt_iso', 'weather_icon', 'city_id'], axis=1, inplace=True)

In [114]:
weather_data.head(3)

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description
dt,,,,,,,,,,,,,,,
2013-10-01 02:00:00,Valencia,299.150,299.150,299.150,1008,61,5,290,0.0,0.0,0.0,20,801,clouds,few clouds
2013-10-01 03:00:00,Valencia,298.150,298.150,298.150,1009,65,4,250,0.0,0.0,0.0,20,801,clouds,few clouds
2013-10-01 04:00:00,Valencia,296.161,296.161,296.161,1009,71,4,269,0.0,0.0,0.0,10,800,clear,sky is clear


In [116]:
weather_data.index.min(),weather_data.index.max() 

(Timestamp('2013-10-01 02:00:00'), Timestamp('2019-08-26 02:00:00'))

In [118]:
#slice starting from 2015
weather_data = weather_data['2015':]

In [126]:
weather_data.to_csv(save_path + 'weather_2015_2019.csv')

In [127]:
!ls -l ./processed

total 60408
-rw-r--r--  1 ns  staff   1518679  7 Oct 16:18 Actual-demand-forecast-demand-2015-2019.csv
-rw-r--r--  1 ns  staff   5329580  7 Oct 18:04 generation-production-type-2015-2019.csv
-rw-r--r--  1 ns  staff   1583537  7 Oct 18:16 solar_wind_forecasts.csv
-rw-r--r--  1 ns  staff  21046004  7 Oct 18:49 weather_2015_2019.csv
